# Семинар 17.

# Линейное программирование. Симплекс-метод

## Текущее место в программе

- Безусловная оптимизация
    - ~~Одномерная минимизация~~
    - ~~Градиентный спуск~~
    - ~~Метод Ньютона и квазиньютоновские методы~~
    - ~~Метод сопряжённых градиентов~~
- Условная оптимизация
    - <span style="color:red">Линейное программирование: симплекс-метод</span>
    - Линейное программирование: прямо-двойственный метод
    - Методы проекции градиента и условного градиента
    - Методы штрафов и барьеров
    - Метод модифицированной функции Лагранжа
    - Последовательное квадратичное программирование
- Разные темы
    - ~~Задача наименьших квадратов~~
    - Проксимальные методы
    - Оптимальные методы и нижние оценки
    - Зеркальный спуск
    - Обзор стохастических методов

## Постановка задачи

- Стандартная форма
\begin{equation*}
\begin{split}
&\min_x c^{\top}x \\
\text{s.t. } & Ax \leq b\\
& x_i \geq 0, \; i = 1,\dots, n
\end{split}
\end{equation*}
- Каноническая форма
\begin{equation*}
\begin{split}
&\min_x c^{\top}x \\
\text{s.t. } & Ax = b\\
& x_i \geq 0, \; i = 1,\dots, n
\end{split}
\end{equation*}

### Преобразование постановок

- $Ax = b \equiv 
\begin{cases}
Ax \leq b\\
Ax \geq b
\end{cases}
$
- $Ax \leq b \equiv 
\begin{cases}
Ax + z =  b\\
z \geq 0
\end{cases}$ 
- свободная переменная $x \equiv 
\begin{cases}
x = u - v\\
u \geq 0, \; v \geq 0
\end{cases}$

## Приложения

- Потоки в сетях
- 

### Примеры

## Алгебраическая vs. геометрическая точка зрения

### Определения

### Всегда ли есть вершины?

### Где искать решение?

### А есть ли решение?

## Симплекс-метод

### Схема

## Реализация

## Узкие места

## Сложность

- Было показано, что в худшем случае время работы симплекс-метод <span style="color:red"> **экспоненциально**</span> зависит от размерности задачи!
- Однако на практике сложность чаще всего пропорциональна количеству ограничений и симплекс-метод сходится быстро
- Почему это так до конца не ясно до сих пор

### Пример Klee, Minty (1972) [[1]](https://en.wikipedia.org/wiki/Klee–Minty_cube)

В задаче
\begin{equation*}
\begin{split}
& \min_x 2^{n-1}x_1 + 2^{n-2}x_2 + \dots + 2x_{n-1} + x_n\\
\text{s.t. } & x_1 \leq 5\\
& 4x_1 + x_2 \leq 25\\
& 8x_1 + 4x_2 + x_3 \leq 125\\
& \ldots\\
& 2^n x_1 + 2^{n-1}x_2 + 2^{n-2}x_3 + \ldots + x_n \leq 5^n\\
& x \geq 0
\end{split}
\end{equation*}
начиная с точки $x_0 = 0$ и следуя симплекс-методу, необходимо обойти $2^n - 1$ вершину.

**Упражнение**: решите эту задачу для $n = 2$ и $n = 3$, и обобщите результат, получив решение для произвольного $n$.